# Metanno demo

In the `recipes/ner.py` file is the main class of the application, it contains everything that allows us to control the application, that is what should be rendered, and how we should react to events.

The `select_editor_state` function returns a json object based on the view that calls it.  
In this example, we have three editors: one to render the annotations on the text, one to render them as an array, and one to view the list of documents.

The `handle_...` functions define how the application should react to a user event.  
The `@produce` decorator is used to allow the application to track state mutations in these functions.  
For instance, look at how we handle a span hover with the `handle_enter_span` and `handle_leave_span` functions.
To force the execution of a function only in the frontend or kernel, the `@kernel_only` and `@frontend_only` decorators must be used (note that we only use the `@frontend_only` decorator to keep the span hover logic in the browser). 

To start the application, run each cell of the notebook.

You can change the layout of these views by right-clicking on them, clicking on `Detach` and moving them around.

In [ ]:
from metanno.recipes.ner import NERApp, colors
from metanno.connectors import BratDataConnector

In [ ]:
data = BratDataConnector("./dataset/", overwrite_ann=True)

labels = sorted(set([ent["label"] for doc in data.load() for ent in doc["entities"]]))
keys = {"modality": "m", "experiencer": "x", "time": "t"}
for label in labels:
    keys[label] = next(letter for letter in label.lower() if letter not in keys.values())
app = NERApp(
    data=data,
    suggester=None,
    scheme={
        "labels": [
            {"name": label, "color": colors[i], "key": keys[label], "alias": label}
            for i, label in enumerate(labels)
        ],
        "attributes": [{
            "name": "modality",
            "kind": "text",
            "key": "m",
            "color": "lightgrey",
            "choices": ["factual", "negated", "conditional", "counterindication", "uncertain", "suggested"]
        }, {
            "name": "experiencer",
            "kind": "text",
            "key": "x",
            "color": "lightgrey",
            "choices": ["self", "family", "other"],
        }, {
            "name": "time",
            "kind": "text",
            "key": "t",
            "color": "lightgrey",
            "choices": ["present", "past", "future"],
        }, {
            "name": "concept",
            "kind": "text",
            "color": "lightgrey",
            "choices": [f"C{n:04}" for n in range(10000)],
        }],
    },
)

In [ ]:
app.span_editor("text")

In [ ]:
app.table_editor("docs")

In [ ]:
app.table_editor("entities")

In [ ]:
# You can call the app methods from Python
# and observe the changes directly in your browser
app.change_doc("doc-2.txt")

In [ ]:
# and even mutate any part of the state directly
app.state["doc_id"] = 0  # set to the first doc

In [ ]:
# Inspect the state
app.state